<a href="https://colab.research.google.com/github/akmhel/Team-Go/blob/master/Recommdatxn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import pandas_profiling
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
tfidf = TfidfVectorizer(stop_words='english')

In [143]:
post_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/posts.csv')
post_data2 = post_data.copy()
post_data2.head()

,id,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id
0,1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,8/25/2019 19:39,8/28/2019 11:30,1,NaN,NaN,NaN
1,2,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,8/25/2019 19:44,8/28/2019 11:30,1,NaN,NaN,NaN
2,4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,8/25/2019 19:50,8/28/2019 11:30,1,NaN,NaN,NaN
3,6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,8/25/2019 19:51,8/28/2019 11:30,1,NaN,NaN,NaN
4,7,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,8/25/2019 19:51,8/28/2019 11:30,1,NaN,NaN,NaN


In [144]:
post_data2.dtypes

id              int64
user_id         int64
title          object
content        object
tags           object
slug           object
created_at     object
updated_at     object
image          object
status_id     float64
action         object
post_id       float64
dtype: object

In [145]:
user_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/users.csv')
user_data2 = user_data.copy()
user_data2.head()

,user_id,name,username,email,image,provider,provider_id,password,remember_token,created_at,updated_at,short_bio
0,1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com,/storage/oluwaseyieniayomi/images//5lxMjdbOVca...,google,1.008920e+20,NaN,79kIhqDiWzqx1i3eFpGMpxfLdPVIvSfv8A3vN2PNEl4ig9...,8/1/2019 17:57,8/1/2019 17:57,Software Developer | DevOPs Engineer
1,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,/storage/2/images//DB7axhFJpFUzPuCNPRMQ6X1iNwb...,google,1.055840e+20,NaN,Rz08FtDtdyAp8Cm5XJq5XumgZhEqytSu5MxxyHZYlHBhd0...,8/1/2019 18:05,8/1/2019 18:05,Web Developer
2,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,https://lh3.googleusercontent.com/-FQgpN5l_UoY...,google,1.012350e+20,NaN,ZxNMYG05B6RQuSWaJlqlc8bGbPyFxNGNnNf5D86biFix4G...,8/1/2019 20:55,8/1/2019 20:55,Front End Developer | React Developer | Mentor...
3,4,Oluwaseyi Oluwapelumi,nathan,nathanoluwaseyi@gmail.com,/storage/4/images/thumbnail/TLWSEBK7C-UMECCGUP...,google,1.072330e+20,NaN,uxbOXr4Cw8tJyilWDKQRhi7TjiV7gBzjMnX0cb4i3ndFu6...,8/2/2019 16:52,8/2/2019 16:52,| Software Developer | DevOps Engineer | @linu...
4,5,PoRH,lamar,paulchibiukeigweze@gmail.com,/storage/5/images/thumbnail/images (2)_small_1...,google,1.034320e+20,NaN,EeCB77oXHo3zdZgYTVhWlw2EyHkTtLSJb2PjWFF9RuG7H1...,8/3/2019 11:33,8/3/2019 11:33,I Am lamar and you don't think am real?


In [146]:
user_data2.dtypes

user_id             int64
name               object
username           object
email              object
image              object
provider           object
provider_id       float64
password          float64
remember_token     object
created_at         object
updated_at         object
short_bio          object
dtype: object

In [0]:
df = pd.merge(post_data2, user_data2, on='user_id')

In [148]:
metadata = df.copy()
df.head()

,id,user_id,title,content,tags,slug,created_at_x,updated_at_x,image_x,status_id,action,post_id,name,username,email,image_y,provider,provider_id,password,remember_token,created_at_y,updated_at_y,short_bio
0,1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,8/25/2019 19:39,8/28/2019 11:30,1,NaN,NaN,NaN,jason obiako,obiako4u53,obiako4u53@gmail.com,https://lh3.googleusercontent.com/-Gnb2o4SA6HU...,google,1.090290e+20,NaN,TitXROeUlC4a4c9VVpaIhfbAu5dPWVC2PrN5lLufalPFbx...,8/20/2019 22:28,8/20/2019 22:28,I am a person of Growth mindset. I believe i c...
1,2,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,8/25/2019 19:44,8/28/2019 11:30,1,NaN,NaN,NaN,Kolawole John,kolawolejohn,kolawole.oajohn@gmail.com,/storage/kolawolejohn/images//h3OofJbAD9cW5Ro9...,google,1.133160e+20,NaN,GeEAAszJ1jjHBXZWwv0ujB1arkHJVMDveiRZSCSfRV4e0Q...,8/19/2019 21:42,8/19/2019 21:42,"I am Kolawole Adebayo Ogunfowokan, I am a trai..."
2,4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,8/25/2019 19:50,8/28/2019 11:30,1,NaN,NaN,NaN,Oladele Makanjuola,Oladele,makanjuolaoladele@gmail.com,/storage/Oladele/images//irm5vHnrgL2XD3bvmhZR5...,google,1.144720e+20,NaN,ArBKC0Ons7Xy3G8llXn6MqcwiDKZxOFPibMlzzY1Lau4Yk...,8/19/2019 12:31,8/19/2019 12:31,Front-End enthusiast
3,6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,8/25/2019 19:51,8/28/2019 11:30,1,NaN,NaN,NaN,Emmanuel ulaka,emmanuel.ulaka16,emmanuel.ulaka16@gmail.com,/storage/1787/images//8RueyRdHpIdJdBZ5I7DWpU7i...,google,1.104470e+20,NaN,B56rhxElOxeJ1BPqj3qJsVz6TzVuc5S6hrOKlpQ1L1fgfr...,8/20/2019 0:45,8/20/2019 0:45,NaN
4,11,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,html,my-task-2-649,8/25/2019 19:57,8/28/2019 11:30,1,NaN,NaN,NaN,Emmanuel ulaka,emmanuel.ulaka16,emmanuel.ulaka16@gmail.com,/storage/1787/images//8RueyRdHpIdJdBZ5I7DWpU7i...,google,1.104470e+20,NaN,B56rhxElOxeJ1BPqj3qJsVz6TzVuc5S6hrOKlpQ1L1fgfr...,8/20/2019 0:45,8/20/2019 0:45,NaN


**Keeps Only Column We'll need**

In [149]:
keep_cols = ['id', 'user_id', 'title', 'content', 'name', 'username']
metadata = metadata[keep_cols]

# Drop row with null title
null_title = metadata[metadata['title'].isna()].index
metadata.drop(index = null_title, inplace = True)

metadata.reset_index(drop = True, inplace = True)

print(metadata.shape)
metadata.head()

(876, 6)


,id,user_id,title,content,name,username
0,1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,jason obiako,obiako4u53
1,2,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Kolawole John,kolawolejohn
2,4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,Oladele Makanjuola,Oladele
3,6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,Emmanuel ulaka,emmanuel.ulaka16
4,11,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,Emmanuel ulaka,emmanuel.ulaka16


In [150]:
metadata.shape

(876, 6)

In [151]:
metadata.describe()

,id,user_id
count,876.000000,876.000000
mean,516.808219,1061.639269
std,278.648058,788.070779
min,1.000000,1.000000
25%,278.750000,338.000000
50%,523.500000,944.000000
75%,754.250000,1836.250000
max,1007.000000,2297.000000


In [152]:
metadata.isnull().sum()

id          0
user_id     0
title       0
content     0
name        0
username    0
dtype: int64

In [0]:
metadata.dropna(axis=1, inplace=True)

In [154]:
metadata.isnull().sum

<bound method DataFrame.sum of         id  user_id  title  content   name  username
0    False    False  False    False  False     False
1    False    False  False    False  False     False
2    False    False  False    False  False     False
3    False    False  False    False  False     False
4    False    False  False    False  False     False
5    False    False  False    False  False     False
6    False    False  False    False  False     False
7    False    False  False    False  False     False
8    False    False  False    False  False     False
9    False    False  False    False  False     False
10   False    False  False    False  False     False
11   False    False  False    False  False     False
12   False    False  False    False  False     False
13   False    False  False    False  False     False
14   False    False  False    False  False     False
15   False    False  False    False  False     False
16   False    False  False    False  False     False
17   False    F

In [155]:
metadata.describe()

,id,user_id
count,876.000000,876.000000
mean,516.808219,1061.639269
std,278.648058,788.070779
min,1.000000,1.000000
25%,278.750000,338.000000
50%,523.500000,944.000000
75%,754.250000,1836.250000
max,1007.000000,2297.000000


In [156]:
pandas_profiling.ProfileReport(metadata)

Number of variables,6
Number of observations,876
Total Missing (%),0.0%
Total size in memory,41.1 KiB
Average record size in memory,48.1 B
Numeric,2
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [157]:
metadata.corr(method = 'pearson')

,id,user_id
id,1.000000,-0.165137
user_id,-0.165137,1.000000


In [158]:
metadata.corr(method = 'kendall')

,id,user_id
id,1.000000,-0.104611
user_id,-0.104611,1.000000


In [159]:
metadata.columns

Index(['id', 'user_id', 'title', 'content', 'name', 'username'], dtype='object')

In [0]:
tfidf_matrix = tfidf.fit_transform(metadata['content'])

In [162]:
tfidf_matrix.shape

(876, 5122)

In [0]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]
  return metadata['title'].iloc[movie_indices]

In [165]:
get_recommendations('What i have learnt so far on HTML')

265                              third
308            WHAT I'VE LEARNT SO FAR
775                             TASK 2
776                             TASK 2
141                      CV Using html
261                               TEst
237           Startng task 2 Completed
28     WHAT I HAVE DONE SO FAR ON HTML
16                  What I done so far
51        Lucid post to start.ng Task1
Name: title, dtype: object

In [166]:
get_recommendations('Welcome to Lucid')

571                    pre- intership on startng
590                                        ochem
464                      my 2nd task on start.ng
465                      my 2nd task on start.ng
615                                My First Post
582                               Test an image.
449         Converting Cv to Html StartNg Task 2
502                      Journey round the world
504                               Journey so far
574    THE SKY IS MY STEPPING STONE WITH STARTNG
Name: title, dtype: object

In [168]:
metadata.title

0                      What i have learnt so far on HTML
1                                       HTML BEGINS HERE
2                                My Laziness In The Open
3                                              MY TASK 2
4                                              MY TASK 2
5                                                 Task 2
6                                Daily Log - CLASS DAY 1
7                                     My Journey on HTML
8                                            Please note
9                                     My Journey on HTML
10                                    My learning so far
11                               StartNG HTML Exposition
12                                     StartNG HTML task
13                             On StartNG Pre-Internship
14                                                   why
15                         HNG internship journey so far
16                                    What I done so far
17                             